In [1]:
import ijson
import pandas as pd
import numpy as np
import re
import os
import warnings
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

# Chemin vers le fichier JSON volumineux dans Google Drive
file_path = 'offres_emploi.json'

# Lire le fichier JSON par morceaux
def read_large_json(file, chunksize=10000):
    """
    Cette fonction lit un fichier JSON volumineux par morceaux de taille spécifiée.

    Args:
    file (str): Le chemin du fichier JSON.
    chunksize (int): Le nombre de lignes par morceau.

    Yields:
    list: Une liste d'objets JSON correspondant à un morceau du fichier.
    """
    with open(file, 'r', encoding='utf-8') as f:
        objects = ijson.items(f, 'item')
        chunk = []
        for i, obj in enumerate(objects):
            chunk.append(obj)
            if (i + 1) % chunksize == 0:
                yield chunk
                chunk = []
        if chunk:
            yield chunk

# Fonction de nettoyage des données
def clean_data(df):
    """
    Cette fonction nettoie les données d'un DataFrame en supprimant les lignes avec des valeurs manquantes
    dans certaines colonnes et en transformant certaines colonnes.

    Args:
    df (pd.DataFrame): Le DataFrame à nettoyer.

    Returns:
    pd.DataFrame: Le DataFrame nettoyé.
    """
    # Fonction pour transformer la colonne dureeTravailLibelle
    def transform_duree_travail_libelle(value):
        if isinstance(value, str):
            numbers = re.findall(r'\d{2,}', value)  # Recherche d'un nombre entier avec au moins deux chiffres
            if numbers:
                return int(numbers[0])
            elif "temps plein" in value:
                return 35
            elif "temps partiel" in value:
                return 24
            else:
                 return 0
        else:
            return 0
                
    # Appliquer la transformation à dureeTravailLibelle
    df['dureeTravailLibelle'] = df['dureeTravailLibelle'].apply(transform_duree_travail_libelle)

#     # Calculer la moyenne de dureeTravailLibelle
#     duree_moyenne = df['dureeTravailLibelle'].mean()

#     # Remplacer les valeurs manquantes de dureeTravailLibelle par la moyenne
#     df['dureeTravailLibelle'].fillna(duree_moyenne, inplace=True)

    # Fonction pour extraire le salaire en float
    def extract_salaire(salaire, duree):
        """
        Extrait et calcule le salaire à partir de la chaîne de salaire donnée.
        
        Args:
        salaire (str): La chaîne de salaire à extraire et à calculer.
        duree (int): La durée de travail associée pour le calcul (en heures).
        
        Returns:
        float: Le salaire extrait et calculé en float, arrondi à 2 décimales, ou la valeur existante si la chaîne de salaire n'est pas valide.
        
        Note:
        - Si la chaîne de salaire est vide ou n'est pas une chaîne valide, la valeur de salaire fournie est retournée.
        - La fonction prend en compte les différentes unités de salaire (horaire, mensuel, annuel, etc.) et effectue les calculs appropriés en fonction de la durée de travail.
        """
        
        # Vérifier si la valeur de salaire est une chaîne
        if isinstance(salaire, str):
            # Capturer à la fois les entiers et les nombres décimaux en utilisant les expressions régulières
            montants = [float(m) for m in re.findall(r'\d+\.?\d*', salaire) if len(m.split('.')[0]) >= 2]
            montants_horaire = [float(m) for m in re.findall(r'\d+\.?\d*', salaire) if len(m.split('.')[0]) >= 1]
            if 'Horaire' in salaire:
                # Calculer le salaire horaire
                if len(montants_horaire) >= 2 and 13.0 in montants_horaire:
                    return round(((sum(montants_horaire[:2]) / 2) * 13 / 12) * duree * 4, 2)
                elif len(montants_horaire) == 1 and 13.0 in montants_horaire:
                    return round((montants_horaire[0] * 13.0 / 12) * duree * 4, 2)
                elif len(montants_horaire) >= 2:
                    return round((sum(montants_horaire[:2]) / 2) * duree * 4, 2)
                elif len(montants_horaire) == 1:
                    return round(montants_horaire[0] * duree * 4, 2) 
            elif 'Mensuel' in salaire or 'Cachet' in salaire:
                # Calculer le salaire mensuel ou par cachet
                if len(montants) >= 2 and 13 in montants:
                    return round((sum(montants[:2]) / 2) * 13.0 / 12, 2)
                elif len(montants) == 1 and 13.0 in montants:
                    return round(montants[0] * 13.0 / 12, 2)
                elif len(montants) >= 2:
                    return round(sum(montants[:2]) / 2, 2)
                elif len(montants) == 1:
                    return round(montants[0], 2)
            elif 'Annuel' in salaire:
                # Calculer le salaire annuel
                montant = montants[0] if montants else 0
                return round(montant / 12, 2)
        # Si la valeur de salaire n'est pas une chaîne valide, retourner la valeur existante
        return salaire

#     # Remplacer les valeurs manquantes par None
#     df['salaire_libelle'] = df['salaire_libelle'].apply(lambda x: None if pd.isna(x) else x)
#     df['entreprise_nom'] = df['entreprise_nom'].apply(lambda x: None if pd.isna(x) else x)
#     # Remplacer les valeurs manquantes de dureeTravailLibelle par la moyenne
#     df['entreprise_nom'].fillna("None", inplace=True)
#     df['salaire_libelle'].fillna("None", inplace=True)
    
    # Appliquer la fonction d'extraction du salaire
    df['salaire_libelle'] = df.apply(lambda row: extract_salaire(row['salaire_libelle'], row['dureeTravailLibelle']), axis=1)

    # Transformer la colonne lieuTravail_libelle pour garder uniquement la partie après le tiret
    df['lieuTravail_libelle'] = df['lieuTravail_libelle'].apply(lambda x: x.split(' - ')[1].strip() if ' - ' in x else x.strip())
    
#     # Remplacer les valeurs 'None' par np.nan
#     df['salaire_libelle'] = df['salaire_libelle'].replace('None', np.nan)
    
    # Convertir la colonne 'salaire_libelle' en float64
    df['salaire_libelle'] = df['salaire_libelle'].astype(float)

    return df

# Fonction de conversion des types de données
def convert_data_types(df):
    """
    Cette fonction convertit les types de données dans un DataFrame.

    Args:
    df (pd.DataFrame): Le DataFrame à convertir.""

    Returns:
    pd.DataFrame: Le DataFrame avec les types de données convertis.
    """
    # Transformer la colonne dateCreation au format "DD-MM-YYYY"
    df['dateCreation'] = pd.to_datetime(df['dateCreation'], errors='coerce').dt.strftime('%d-%m-%Y')

    return df

# Sauvegarde des données nettoyées dans un fichier CSV
output_file = 'cleaned_offres_emploi.csv'

def save_to_csv(df, output_file, first_chunk):
    """
    Cette fonction sauvegarde un DataFrame dans un fichier CSV.

    Args:
    df (pd.DataFrame): Le DataFrame à sauvegarder.
    output_file (str): Le chemin du fichier CSV.
    first_chunk (bool): Indique s'il s'agit du premier morceau à sauvegarder.
    """
    if first_chunk:
        df.to_csv(output_file, index=False, mode='w', encoding='utf-8', sep=',')
    else:
        df.to_csv(output_file, index=False, mode='a', encoding='utf-8', sep=',', header=False)

# Traitement des données par morceaux
first_chunk = True
total_documents_transformed = 0
total_documents_stored = 0

# Lire et traiter chaque morceau de données
for chunk in read_large_json(file_path):
    # Convertir le morceau en DataFrame
    df_chunk = pd.DataFrame(chunk)

#     print(f"Chunk de départ : {len(df_chunk)} documents")
    total_documents_transformed += len(df_chunk)
    try:
        # Nettoyer les données
        df_chunk = clean_data(df_chunk)
        # Convertir les types de données
        df_chunk = convert_data_types(df_chunk)

        # Renommer les colonnes
        df_chunk.rename(columns={
            'lieuTravail_libelle': 'Address',
            'intitule': 'Job',
            'description': 'Description',
            'dateCreation': 'Date',
            'typeContrat': 'Type of contract',
            'salaire_libelle': 'Salary',
            'origineOffre_urlOrigine': 'Link',
            'entreprise_nom': 'Company'
        }, inplace=True)

        # Conserver uniquement les colonnes spécifiées et dans l'ordre souhaité
        columns_to_keep = ['Job', 'Company', 'Address', 'Date', 'Link', 'Salary', 'Description']
        df_chunk = df_chunk[columns_to_keep]

        # Supprimer les doublons
        df_chunk.drop_duplicates(inplace=True)

        total_documents_stored += len(df_chunk)
    except KeyError as e:
        print(f"Erreur KeyError : {e}")
        continue

    # Sauvegarder les données nettoyées dans un fichier CSV
    save_to_csv(df_chunk, output_file, first_chunk)
    first_chunk = False
#     print(f"Documents stockés après ce chunk : {total_documents_stored}")

print(f"Nettoyage et sauvegarde des données terminés. Nombre total de documents transformés : {total_documents_transformed}")
# print(f"Nombre total de documents réellement stockés : {total_documents_stored}")

# Lire le fichier CSV et afficher le nombre de documents
df_csv = pd.read_csv(output_file)
print(f"Nombre total de documents dans le fichier CSV : {len(df_csv)}")

# Supprimer les doublons
df_csv.drop_duplicates(inplace=True)

print(f"Nombre total de documents dans le fichier CSV après suppression des doublons : {len(df_csv)}")

# Afficher la taille du fichier
file_size_mb = os.path.getsize(output_file) / (1024 * 1024)
print(f"Taille du fichier {output_file} : {file_size_mb:.2f} Mo")

Nettoyage et sauvegarde des données terminés. Nombre total de documents transformés : 21233
Nombre total de documents dans le fichier CSV : 21233
Nombre total de documents dans le fichier CSV après suppression des doublons : 21233
Taille du fichier cleaned_offres_emploi.csv : 27.87 Mo
